In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
from selenium.common.exceptions import NoSuchElementException


# Iteration 1. Scraping Vasastan succesfully (although each page is being scrubbed, and that is taking a long long time to process)

## This solution is very redudant to acquire some more features, although, it takes too long to run atm...


In [15]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

vasastan_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=925970"

path=r"C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/chromedriver.exe"
driver = webdriver.Chrome(executable_path = path, options=options)

def hemnet_scraping(url, num_entries):
    try:
        driver.get(vasastan_url)
        time.sleep(2)
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()

        except(org.openqa.selenium.StaleElementReferenceException):
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()

        time.sleep(2)
        elements = driver.find_elements_by_class_name("sold-property-link")
        housing_dict = {}
        #50 entries per page, so at the count of 50, the page has to change
        entries_per_page = 50
        outer_list = list(range(0, num_entries, entries_per_page)) 

        for outer in outer_list:
            print(outer)
            elements = driver.find_elements_by_class_name("sold-property-link")
            for i in range(0,entries_per_page-1):
                print("DETTA I ÄR DET JUST NU: ", i , "\n", "\n")
                print("HUr lång är elements? : ", len(elements))
                housing_info_text = get_features_from_hemnet(i, elements)
                housing_dict[i+outer] = attribute_extractor(housing_info_text)
                time.sleep(2)
                driver.close();
                time.sleep(2)
                driver.switch_to.window(driver.window_handles[0]);
                time.sleep(2)
            #Identify next page element and scroll down to it
            next_page = driver.find_element_by_xpath('//*[@class="next_page hcl-button hcl-button--primary hcl-button--full-width"]')
            #driver.execute_script("arguments[0].scrollIntoView();", next_page) 
            time.sleep(1)
            #Change page
            next_page.click()
    except Exception as e:
        print(e, "caught at: ", outer, "" , i) 
        return housing_dict
    
    return housing_dict


def get_features_from_hemnet(index, elements):
    time.sleep(1)
    elements[index].click()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)

    adress = driver.find_element_by_xpath("//html/body/div[3]/div[2]/div/div[1]/h1").get_attribute('textContent')
    adress = adress.replace("\n  Slutpris\n  ", "")

    price = driver.find_element_by_class_name("sold-property__price-value")
    price = price.text

    time_sold = driver.find_element_by_xpath("//html/body/div[3]/div[2]/div/div[1]/p/time").get_attribute("textContent")
    
    print(time_sold)
    attributes = driver.find_element_by_class_name("sold-property__attributes")
    attributes_text = attributes.text

    stats = driver.find_element_by_class_name("sold-property__price-stats")
    stats_text = stats.text

    housing_info_text = "Pris\n"+price+ "\nadress\n"+ adress + "Datum såld:" + time_sold + attributes_text + "\n" + stats_text
    return housing_info_text 

def attribute_extractor(text):
    
    #text = text_element_list.text
    word_list = []
    while text.find("\n") != -1:
        cut_off = text.find("\n")
        word_list.append(text[0:cut_off])
        text = text[cut_off+1:len(text)]   
    word_list.append(text)

    attribute_list = []
    attribute_values = []


    for i in range(0, len(word_list)):
        if i % 2 == 0:
            attribute_list.append(word_list[i])
        else:
            attribute_values.append(word_list[i])

    res = dict(zip(attribute_list, attribute_values))
    return res

#How many entries we want to extract
num_entries = 2000
vasastan_dict = hemnet_scraping(vasastan_url, num_entries)

vasastan_df = pd.DataFrame(vasastan_dict)
vasastan_df = vasastan_df.T

vasastan_df.to_csv(index=False)

C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == '__main__':
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


0
DETTA I ÄR DET JUST NU:  0 
 

HUr lång är elements? :  50


C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead



      10 januari 2022
    
DETTA I ÄR DET JUST NU:  1 
 

HUr lång är elements? :  50

      8 januari 2022
    
DETTA I ÄR DET JUST NU:  2 
 

HUr lång är elements? :  50

      7 januari 2022
    
DETTA I ÄR DET JUST NU:  3 
 

HUr lång är elements? :  50

      6 januari 2022
    
DETTA I ÄR DET JUST NU:  4 
 

HUr lång är elements? :  50

      6 januari 2022
    
DETTA I ÄR DET JUST NU:  5 
 

HUr lång är elements? :  50

      5 januari 2022
    
DETTA I ÄR DET JUST NU:  6 
 

HUr lång är elements? :  50

      5 januari 2022
    
DETTA I ÄR DET JUST NU:  7 
 

HUr lång är elements? :  50

      4 januari 2022
    
DETTA I ÄR DET JUST NU:  8 
 

HUr lång är elements? :  50

      4 januari 2022
    
DETTA I ÄR DET JUST NU:  9 
 

HUr lång är elements? :  50

      3 januari 2022
    
DETTA I ÄR DET JUST NU:  10 
 

HUr lång är elements? :  50

      26 december 2021
    
DETTA I ÄR DET JUST NU:  11 
 

HUr lång är elements? :  50

      23 december 2021
    
DETTA I ÄR DET JUST 

KeyboardInterrupt: 

# Iteration 2. Scraping only sold page at a time

In [10]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')

vasastan_url = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=925970"

path=r"C:/Users/Anton/Documents/Anton_Gollbo/Skolarbete/projects/Hemnet_Housing/chromedriver.exe"
driver = webdriver.Chrome(executable_path = path, options=options)

def attribute_extractor(text):
    
    labels = ["adress", "location", "size:rooms", "fee", "features", "sale_price", "sold_date","value_dev", "ppsqm"]
    accomodation_features = [0] * len(labels)
    index_track = 0
    while(text.find("\n") != -1):
        if(text.find("hiss") == -1 and text.find("balkong") == -1 and index_track == 0):
            accomodation_features[4] = "null"
        if(text.find("%") == -1 and index_track == 0):
            accomodation_features[6] = "null"
        if(text.find("balkong") != -1 and text.find("balkong") != -1):
            pass
        
        index = text.find("\n")
        extracted_word= text[0:index]
        accomodation_features[index_track] = extracted_word
        text = text[index+1:len(text)]
        
        index_track = index_track +1
        
    #accomodation_features[-1] = text
    
    housing_info_dict = dict(zip(labels,accomodation_features))
    return housing_info_dict


def hemnet_scraping(url, num_entries):
    entries_per_page = 50
    outer_list = list(range(0, num_entries, entries_per_page)) 
    big_housing_dict = {}
    
    driver.get(vasastan_url)
    time.sleep(2)
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()

    except(org.openqa.selenium.StaleElementReferenceException):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="hcl-button hcl-button--primary"]'))).click()
    
    for adding_number in outer_list:

        elements = driver.find_elements_by_xpath('//*[@class ="sold-property-listing qa-sale-card"]')

        for i in range(0,len(elements)):
            big_housing_dict[i+adding_number] = attribute_extractor(elements[i].text)
        
        #Identify next page element and scroll down to it
        next_page = driver.find_element_by_xpath('//*[@class="next_page hcl-button hcl-button--primary hcl-button--full-width"]')
        #driver.execute_script("arguments[0].scrollIntoView();", next_page) 
        time.sleep(1)
        #Change page
        next_page.click()
    
    return big_housing_dict
        
       
    

vasastan_dict = hemnet_scraping(vasastan_url, 50) 
vasastan_df = pd.DataFrame(vasastan_dict)
vasastan_df = vasastan_df.T



C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if __name__ == '__main__':
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


In [11]:
vasastan_df

,adress,location,size:rooms,fee,features,sale_price,sold_date,value_dev,ppsqm
0,Rådmansgatan 75,"Vasastan, Stockholms kommun",35 m² 1 rum,2 085 kr/mån,Hiss,Slutpris 4 400 000 kr,Såld 10 januari 2022,+11 %,0
1,"Dalagatan 70, 4tr","Vasastan, Stockholms kommun",80 m² 4 rum,3 086 kr/mån,Balkong,Slutpris 9 800 000 kr,Såld 8 januari 2022,+11 %,0
2,Völundsgatan 11,"Vasastaden, Stockholms kommun",38 m² 2 rum,2 147 kr/mån,Hiss,Slutpris 4 600 000 kr,Såld 7 januari 2022,±0 %,0
3,"Torsgatan 40a, 3tr ög","Vasastan - S:t Eriksplan, Stockholms kommun",54 m² 2 rum,3 082 kr/mån,Balkong,Slutpris 6 150 000 kr,Såld 6 januari 2022,0,0
4,Ingemarsgatan 3B,"Vasastan, Stockholms kommun",32 m² 2 rum,1 658 kr/mån,Balkong,Hiss,Slutpris 4 300 000 kr,Såld 6 januari 2022,0
5,Vulcanusgatan 13,"Vasastaden, Stockholms kommun",38 m² 2 rum,1 986 kr/mån,Hiss,Slutpris 4 600 000 kr,Såld 5 januari 2022,+15 %,0
6,"Frejgatan 50, 2 tr","Vasastan - Odenplan, Stockholms kommun",47 m² 2 rum,2 553 kr/mån,Slutpris 6 800 000 kr,Såld 5 januari 2022,null,0,0
7,Torsgatan 78,"Vasastaden, Stockholms kommun",47 m² 2 rum,2 454 kr/mån,Slutpris 5 350 000 kr,Såld 4 januari 2022,null,0,0
8,"Upplandsgatan 49, 3 tr","Vasastan, Stockholms kommun",64 m² 3 rum,3 459 kr/mån,Balkong,Hiss,Slutpris 6 200 000 kr,Såld 4 januari 2022,+3 %
9,"Västmannagatan 18, 4 tr","Vasastan, Stockholms kommun","35 m² 1,5 rum",2 108 kr/mån,Balkong,Hiss,Slutpris 4 500 000 kr,Såld 3 januari 2022,+13 %


In [105]:


respekt = {}
for i in range(0,len(elements)):
    respekt[i] = attribute_extractor(elements[i].text)


In [108]:
entries_per_page = 50
outer_list = list(range(0, num_entries, entries_per_page)) 

In [109]:
for i in outer_list:
    print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950


In [144]:

len(labels)

9